In [ ]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=afe41af82c12330183dc850fa0132764a04be88f351e772e189117fd64a5c2cc
  Stored in directory: c:\users\dr zulfiqar\appdata\local\pip\cache\wheels\f5\0c\3a\61b992f7aa85de40f339e6d4970d91dddb103dd0ad6c5d58f2
Successfully built imutils


In [ ]:
import numpy as np 
import cv2
import os
from os import listdir
import shutil
import itertools
import imutils
from tqdm import tqdm
from PIL import Image


In [ ]:
def crop_brain_contour(image, plot=False):
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image


In [ ]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 'glioma':
            cv2.imwrite(folder_name+'glioma/'+str(i)+'.png', img)
        elif imclass == 'meningioma':
            cv2.imwrite(folder_name+'meningioma/'+str(i)+'.png', img)
            
        else:
            cv2.imwrite(folder_name+'pituitary tumor/'+str(i)+'.png', img)
        i += 1

In [ ]:
labels = ['glioma','meningioma','pituitary tumor']

X_train = []
y_train = []
X_test = []
y_test = []

print("Train Set")
print("===========")
image_size = 240
for i in labels:
  # path to dataset
    folderPath = os.path.join('E:/Personal Projects/Multiclass Brain Tumor Classification/Dataset/Split Data 80-20','train',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = crop_brain_contour(img)
        X_train.append(img)
        y_train.append(i)
                
X_train = np.array(X_train)
y_train = np.array(y_train)

print("Test Set")
print("=========")
for i in labels:
  # path to dataset
    folderPath = os.path.join('E:/Personal Projects/Multiclass Brain Tumor Classification/Dataset/Split Data 80-20','test',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = crop_brain_contour(img)
        X_test.append(img)
        y_test.append(i)
                
X_test = np.array(X_test)
y_test = np.array(y_test)

  3%|▎         | 30/1140 [00:00<00:03, 292.06it/s]

Train Set


100%|██████████| 744/744 [00:02<00:00, 288.25it/s]
C:\Users\DR ZULFIQAR\.conda\envs\tf_env\lib\site-packages\ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
 23%|██▎       | 65/286 [00:00<00:00, 312.50it/s]

Test Set


100%|██████████| 186/186 [00:00<00:00, 273.86it/s]
C:\Users\DR ZULFIQAR\.conda\envs\tf_env\lib\site-packages\ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# path to save cropped image
save_new_images(X_train, y_train, folder_name='E:/Personal Projects/Multiclass Brain Tumor Classification/Dataset/Cropped 2/train/')
save_new_images(X_test, y_test, folder_name='E:/Personal Projects/Multiclass Brain Tumor Classification/Dataset/Cropped 2/test/')